In [455]:
import cPickle as pickle
from matplotlib import pyplot as plt
import os
import numpy as np

# Grab Input Data
training_labels_file = '../finetuning/VOC_cropped_warped_train_small.txt'
data_files_path = 'rcnn_features/'

data = []
labels = []
data_files = []
with open(training_labels_file) as f:
    for line in f:
        data_file, label = line.split(' ')
        data_file = data_file.split('/')[-1].split('.')[0] + '.pi'
        try:
            data.append(pickle.load(open(data_files_path + data_file, 'rb')))
            labels.append(int(label))
            data_files.append(data_file)
        except IOError as e:
            pass
data = np.squeeze(np.array(data))
labels = np.array(labels)
data_files = np.array(data_files)
print data.shape, labels.shape, data_files.shape

(909, 200) (909,) (909,)


In [456]:
# Split data into training and test
num_vectors = data.shape[0]
test_ratio = .2
num_test = num_vectors * test_ratio

indices = np.random.permutation(num_vectors)
training_idx, test_idx = indices[num_test:], indices[:num_test]
training_data, test_data = data[training_idx,:], data[test_idx,:]
training_labels, test_labels = labels[training_idx,], labels[test_idx,]

print training_data.shape, test_data.shape
print training_labels.shape, test_labels.shape

(728, 200) (181, 200)
(728,) (181,)


In [457]:
# Define Classifier
# http://scikit-learn.org/stable/modules/grid_search.html
from sklearn import svm, neighbors, linear_model, ensemble, grid_search 

scoring = 'f1_weighted'
# classifier = grid_search.GridSearchCV(svm.SVC(class_weight='auto'), {'kernel':('linear', 'rbf'), 'C':[.001, .01, .1, 1, 10, 100, 1000]}, scoring=scoring)
classifier = ensemble.RandomForestClassifier(max_depth=None, class_weight='subsample')

# classifier = grid_search.GridSearchCV(svm.LinearSVC(loss='l1', class_weight='auto'), {'C':[.001, .01, .1, 1, 10, 100, 1000]}, scoring=scoring)
# classifier = linear_model.RidgeClassifierCV()
# classifier = linear_model.SGDClassifier()
# classifier = neighbors.KNeighborsClassifier()
# classifer = ensemble.AdaBoostClassifier()
# classifer = ensemble.GradientBoostingClassifier()
# classifer = ensemble.ExtraTreesClassifier()

In [458]:
% time
# Train Classifier
classifier.fit(training_data, training_labels);

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 5.96 µs


In [459]:
# Test Classifier
from sklearn.cross_validation import cross_val_score

predicted_labels = classifier.predict(test_data)
print cross_val_score(classifier, training_data, training_labels, scoring=scoring)
print classifier.score(test_data, test_labels) # score is always accuracy

[ 0.57237537  0.56336525  0.60028483]
0.662983425414


In [460]:
import pandas
from sklearn.metrics import confusion_matrix
wrong_test_idx = np.arange(test_labels.shape[0])[predicted_labels != test_labels]
wrong_data_files_idx = test_idx[wrong_test_idx]
wrong_data_files = data_files[wrong_data_files_idx]

wrong_predicted = predicted_labels[predicted_labels != test_labels]
wrong_true = test_labels[predicted_labels != test_labels]

# Labels 0-4 = person, bike, bus, car, motorbike
print "Training Confusion Matrix"
print confusion_matrix(training_labels, classifier.predict(training_data))
print "Test Results"
print "Wrong:", len(wrong_true), '/', len(test_labels), ',', float(len(wrong_true))/len(test_labels)
print confusion_matrix(test_labels, predicted_labels)
print pandas.DataFrame({"Image Index": wrong_data_files, "Predicted": wrong_predicted, "Truth": wrong_true})

Training Confusion Matrix
[[484  33   0   1   0]
 [  3  32   0   0   0]
 [  4   1  27   0   0]
 [  5   1   0  95   0]
 [  2   1   0   0  39]]
Test Results
Wrong: 61 / 181 , 0.337016574586
[[119   9   0   3   0]
 [ 10   0   0   0   0]
 [ 12   0   0   0   0]
 [ 20   1   2   1   0]
 [  4   0   0   0   0]]
    Image Index  Predicted  Truth
0   pict_551.pi          0      3
1   pict_748.pi          2      3
2   pict_534.pi          0      2
3   pict_997.pi          0      3
4   pict_670.pi          0      4
5   pict_500.pi          1      0
6   pict_277.pi          1      0
7   pict_602.pi          1      0
8   pict_712.pi          1      0
9   pict_928.pi          0      1
10  pict_411.pi          0      3
11  pict_371.pi          0      3
12  pict_558.pi          1      3
13  pict_700.pi          0      3
14  pict_962.pi          0      2
15  pict_725.pi          1      0
16  pict_987.pi          0      1
17  pict_779.pi          0      2
18  pict_195.pi          0      3
19  pict_494.pi 